In [35]:
import selenium
import time #time.sleep에 필요
import winsound

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime # 날짜 관련 라이브러리
from datetime import timedelta
from dateutil import relativedelta

# 아래부터는 뤱페이지 로딩 대기를 위해 필요한 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [25]:
options = webdriver.ChromeOptions()
options.add_argument("user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47")

driver = webdriver.Chrome(options=options)

driver.get('https://www.kopis.or.kr/por/stats/perfo/perfoStatsTotal.do?menuId=MNU_000135&searchWord=&searchType=total')

In [26]:
# getMonthRange 함수 개요:
# year와 month를 입력받는다 (2019, 01)
# 해당 month의 시작일자와 마지막일자를 구한다. (1월31일, 2월28일....11월30일, 12월31일)
# yyyy.mm.dd 형식으로 리턴 (2019.01.01)
def getMonthRange(year, month): 
    year = int(year)
    month = int(month)
    this_month = datetime(year=year, month=month, day=1).date()
    next_month = this_month + relativedelta.relativedelta(months=1)
    first_day = this_month
    last_day = next_month - timedelta(days=1)
    month__=format(first_day.month,'02')+'.'
    month__day=month__+format(first_day.day,'02')
    day_left= '{}.{}'.format(year,month__day)
    month__ = format(first_day.month, '02') + '.'
    month__day = month__ + format(last_day.day, '02')
    day_right = '{}.{}'.format(year, month__day)
    return day_left, day_right

In [27]:
this_month = datetime(year=2019, month=1, day=1).date()
print(this_month)

2019-01-01


In [28]:
# searchData 함수 개요 : 
# 검색 범위를 입력받는다 (dayleft, dayright) ex) 2019.01.01, 2019.01.31
# selenium 으로 웹페이지를 컨트롤해서 조회 버튼 클릭
# 결과로 나온 데이터 테이블을 crwaling 해서 result 변수에 담는다.
# result 변수를 리턴해준다.
def searchData(dayleft,dayright) : 
    elem = driver.find_element_by_id('startDate') #검색 시작 날짜
    elem.send_keys(Keys.CONTROL + 'a') # ctrl + a : 커서가 올라간 범위내 모든 텍스트 선택
    elem.send_keys(Keys.BACKSPACE) # backspace 키 입력으로 선택된 부분 clear
    elem.send_keys(dayleft) # 검색시작일자 입력 (2019.01.01)
    elem.send_keys(Keys.ENTER)
    time.sleep(1) # 엔터키 누르고 바로 넘어가면 다음 입력이 안되는 경우가 있어서 1초간 대기

    elem = driver.find_element_by_id('endDate') #검색 마지막 날짜
    elem.send_keys(Keys.CONTROL + 'a')
    elem.send_keys(Keys.BACKSPACE)
    elem.send_keys(dayright)
    elem.send_keys(Keys.ENTER)
    time.sleep(1)

    elem = driver.find_element_by_xpath('//*[@id="search_list"]/div/a') # 클릭버튼 xpath
    elem.click()
    
    # 페이지 로딩을 기다리는 내장함수, 로딩 아이콘이 사라질때 까지 대기
    wait = WebDriverWait(driver, 60).until(
        EC.invisibility_of_element_located((By.XPATH, '//*[@id="progress"]'))
    )

    table = driver.find_element_by_xpath('//*[@id="su_con"]/div[5]/table') # 결과 테이블 xpath
    tbody = table.find_element_by_tag_name('tbody') # 테이블구조 : table > thead, tbody, tfoot> tr > td
    result = ''                                     # 우리가 필요한 데이터는 tbody 에 존재하므로 tbody 선택
    
    for tr in tbody.find_elements_by_class_name("trTotal"): # 테이블 tbody 에 있는 trTotal 행 선택
        txt = tr.find_element_by_tag_name("th").get_attribute("innerText") # th는 가장 좌측에 있는 권역이름(서울, 경기도....)
        txt += ' ' + dayleft + ' ' + dayright
        for td in tr.find_elements_by_tag_name("td")[1:]: # td[0] = 합계, td[1] = 공연건수(건), td[2] = 공연건수(%), td[3] = 개막편수
            txt += ' ' + td.get_attribute("innerText").replace(',', '') # innertext는 괄호와 괄호 사이에 있는 텍스트를 읽는다 <td> innerText </td> 
        result += txt + '\n' # th + td[1], td[2].... td[마지막] 까지 문자열 데이터를 붙여서 결과 함수에 담는다. '\n'은 줄바꿈
    
    #합계부분
    #txt2 = ''
    #for tr in tbody.find_elements_by_class_name('sum.brb_e.bg'):
    #   for td in tr.find_elements_by_tag_name("td"):
    #       txt2 += td.get_attribute("innerText") + ' '
    #   #result += txt2 + '\n'
    #print(result)
    
    return result


In [29]:
# make_year_month 함수 개요:
# 시작연도 입력시 현재 연+월 에서 1달을 뺀 전체 데이터를 리턴
# ex) 2019 입력시 2019년01월부터 현재날짜인 2022년11월 바로 전달까지 리턴 (2019.01~~~~2022.10)
def make_year_month(startyear) :
    nowyear = datetime.now().year
    nowmonth = datetime.now().month
    year_month=[]
    for year in range(startyear,nowyear) :
        for month in range (1,13) :
            year_month.append((year,month))
    for month in range(1,nowmonth) :
        year_month.append((nowyear,month))
    return year_month

make_year_month(2019)

a=make_year_month(2019)
print(a)

[(2019, 1), (2019, 2), (2019, 3), (2019, 4), (2019, 5), (2019, 6), (2019, 7), (2019, 8), (2019, 9), (2019, 10), (2019, 11), (2019, 12), (2020, 1), (2020, 2), (2020, 3), (2020, 4), (2020, 5), (2020, 6), (2020, 7), (2020, 8), (2020, 9), (2020, 10), (2020, 11), (2020, 12), (2021, 1), (2021, 2), (2021, 3), (2021, 4), (2021, 5), (2021, 6), (2021, 7), (2021, 8), (2021, 9), (2021, 10), (2021, 11), (2021, 12), (2022, 1), (2022, 2), (2022, 3), (2022, 4), (2022, 5), (2022, 6), (2022, 7), (2022, 8), (2022, 9), (2022, 10)]


In [8]:
# 연도 입력시 해당월의 전월까지 데이터를 가져오는 ㅂ주분
year_start=2022
year_month_list = make_year_month(year_start)
total_data = []
for ym in year_month_list :
    dayleft, dayright = getMonthRange(ym[0], ym[1])
    result = searchData(dayleft, dayright) # searchdata 함수를 반복 호출해서 result 변수에 담는다.
    result = result.strip().split('\n')
    total_data.append(result)

In [30]:
#total_data = []
print(total_data)

[['서', '울', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '0', '1', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '3', '1', ' ', '3', '9', '9', ' ', '5', '7', '.', '4', ' ', '2', '7', '9', ' ', '5', '1', '.', '8', ' ', '4', '6', '6', '8', ' ', '7', '7', '.', '6', ' ', '6', '7', '1', '6', '9', '6', ' ', '7', '9', '.', '9', ' ', '3', '4', '6', '6', '2', '0', '8', '9', ' ', '8', '1', '.', '7', '\n', '경', '상', '도', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '0', '1', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '3', '1', ' ', '1', '1', '4', ' ', '1', '6', '.', '4', ' ', '9', '6', ' ', '1', '7', '.', '8', ' ', '6', '0', '8', ' ', '1', '0', '.', '1', ' ', '7', '3', '6', '8', '5', ' ', '8', '.', '8', ' ', '4', '2', '6', '2', '6', '7', '6', ' ', '1', '0', '.', '0', '\n', '경', '기', '도', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '0', '1', ' ', '2', '0', '2', '2', '.', '0', '1', '.', '3', '1', ' ', '8', '7', ' ', '1', '2', '.', '5', ' ', '7', '9', ' ', '1', '4', '.', '7', ' ', '3', '7', '6', '

In [ ]:
import pymysql

In [10]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='asdf12', db='mywork', charset='utf8')

In [11]:
db

In [34]:
db.close()

In [9]:
resultrow = result.strip().split('\n')
#print(resultrow)
td = resultrow[0]
td = td.split()
#print(td)
#print(len(resultrow))
for i in range(0, len(resultrow)):
    td = resultrow[i]
    td = td.split()
    print(td)
    #print(len(td))


AttributeError: 'list' object has no attribute 'strip'

In [31]:
def isNumber(n):
    if n != 'NULL':
        try:
            float(n) or int(n)
        except ValueError:
            return False

    return True

def createInsertQuery(tablename : str, data : dict):
    sql= 'INSERT into `' + tablename + '` (`{}`) values ({})'
    key = '`, `'.join(list(data.keys()))
    value=', '.join(str(e) if isNumber(str(e)) else '"'+str(e)+'"' for e in data.values())
    return(sql.format(key,value))
tablename = 'kopis_data'

# data = {
#         'location' : location
#         , 'start_date' : start_date
#         , 'end_date' : end_date
#         , 'perf_count' : perf_count
#         , 'perf_per' : perf_per
#         , 'opening_count' : opening_count
#         , 'opening_per' : opening_per
#         , 'play_count' : play_count
#         , 'play_per' : play_per
#         , 'ticket_sale_count' : ticket_sale_count
#         , 'ticket_sale_per' : ticket_sale_per
#         , 'ticket_sale_amt' : ticket_sale_amt
#         , 'ticket_sale_amt_per' : ticket_sale_amt_per
#     }

In [32]:
dayleft, dayright = getMonthRange('2019', '03')
result = searchData('2019.05.01', '2019.05.31')
print(result)

서울 2019.05.01 2019.05.31 368 58.5 252 52.9 3119 74.9 279087 70.9 6502979 72.8
경상도 2019.05.01 2019.05.31 84 13.4 68 14.3 504 12.1 46123 11.7 1479511 16.6
경기도 2019.05.01 2019.05.31 66 10.5 58 12.2 183 4.4 32703 8.3 520154 5.8
전라도 2019.05.01 2019.05.31 65 10.3 59 12.4 191 4.6 25361 6.4 293865 3.3
충청도 2019.05.01 2019.05.31 38 6.0 33 6.9 145 3.5 9281 2.4 124649 1.4
제주도 2019.05.01 2019.05.31 5 0.8 3 0.6 15 0.4 467 0.1 11750 0.1
강원도 2019.05.01 2019.05.31 3 0.5 3 0.6 7 0.2 670 0.2 4856 0.1



In [33]:
#db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='asdf12', db='mywork', charset='utf8')
resultrow = result.strip().split('\n')
print(resultrow)
tablename = 'kopis_data'
for i in range(0, len(resultrow)):
    td = resultrow[i]
    td = td.split()
    #print(td)
    
    location = td[0]
    start_date = td[1]
    end_date = td[2]
    perf_count = td[3]
    perf_per = td[4]
    opening_count = td[5]
    opening_per = td[6]
    play_count = td[7]
    play_per = td[8]
    ticket_sale_count = td[9]
    ticket_sale_per = td[10]
    ticket_sale_amt = td[11]
    ticket_sale_amt_per = td[12]
    
    data = {
        'location' : location
        , 'start_date' : start_date
        , 'end_date' : end_date
        , 'perf_count' : perf_count
        , 'perf_per' : perf_per
        , 'opening_count' : opening_count
        , 'opening_per' : opening_per
        , 'play_count' : play_count
        , 'play_per' : play_per
        , 'ticket_sale_count' : ticket_sale_count
        , 'ticket_sale_per' : ticket_sale_per
        , 'ticket_sale_amt' : ticket_sale_amt
        , 'ticket_sale_amt_per' : ticket_sale_amt_per
    }
#     query = createInsertQuery(tablename, data)
#     print(query)
#     cursor = db.cursor()
#     cursor.execute(query)
#     db.commit()    
# db.close()



['서울 2019.05.01 2019.05.31 368 58.5 252 52.9 3119 74.9 279087 70.9 6502979 72.8', '경상도 2019.05.01 2019.05.31 84 13.4 68 14.3 504 12.1 46123 11.7 1479511 16.6', '경기도 2019.05.01 2019.05.31 66 10.5 58 12.2 183 4.4 32703 8.3 520154 5.8', '전라도 2019.05.01 2019.05.31 65 10.3 59 12.4 191 4.6 25361 6.4 293865 3.3', '충청도 2019.05.01 2019.05.31 38 6.0 33 6.9 145 3.5 9281 2.4 124649 1.4', '제주도 2019.05.01 2019.05.31 5 0.8 3 0.6 15 0.4 467 0.1 11750 0.1', '강원도 2019.05.01 2019.05.31 3 0.5 3 0.6 7 0.2 670 0.2 4856 0.1']


In [21]:
resultrow


NameError: name 'resultrow' is not defined

In [16]:
db.rollback

<bound method Connection.rollback of <pymysql.connections.Connection object at 0x0000026EE90B7AC0>>

In [95]:
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

cursor = db.cursor()
cursor.execute(query)

db.commit()
db.close()

In [14]:
driver.close()

In [49]:
winsound.Beep(
    frequency=880,  # Hz
    duration=1000  # milliseconds
    )
